# Merge datasets
In this notebook, we link together customer, merchant, transaction, and customer demographic data. This is performed in four key stages:

1. [Join BNPL data (consumers, merchants, transactions)](#join-customermerchanttransaction-data)
2. [Prepare census data for merging](#prepare-external-datasets)
3. [Join BNPL data with census data](#join-abs-and-customer-data)

# Join customer/merchant/transaction data

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
import requests

In [ ]:
# Create a spark session
spark = (
    SparkSession.builder.appName("Data Joining")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "9g") 
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.network.timeout", "600s")
    .getOrCreate()
)

In [ ]:
# Load in datasets
# Load in merchant data (parquet)
merchant = spark.read.parquet("../data/curated/part_1/clean_merchant.parquet")

# Load in merchant fraud (csv)
merchant_fp = pd.read_csv("../data/tables/part_1/merchant_fraud_probability.csv")
merchant_fp = spark.createDataFrame(merchant_fp)

# Load in consumer list (csv)
consumer_cid = pd.read_csv("../data/tables/part_1/tbl_consumer.csv", delimiter="|")
consumer_cid = spark.createDataFrame(consumer_cid)

# Load in consumer fraud (csv)
consumer_fp = pd.read_csv("../data/tables/part_1/consumer_fraud_probability.csv")
consumer_fp = spark.createDataFrame(consumer_fp)

consumer_ud = spark.read.parquet("../data/tables/part_1/consumer_user_details.parquet")

**Prepare customer data**: associate customers with a `user_id`.

In [ ]:
# Joining user id to customers
consumer_cid = consumer_cid.withColumn("postcode", consumer_cid.postcode.cast('string')) # cast postcode to string
consumer = consumer_cid.join(consumer_ud, on = "consumer_id", how = 'left')
consumer_list = consumer.select("user_id", "postcode")
consumer.show(5)

**Join customers and transaction data**

In [ ]:
# Read transaction dataset
transaction1 = spark.read.parquet("../data/tables/part_2")
transaction2 = spark.read.parquet("../data/tables/part_3")
transaction3 = spark.read.parquet("../data/tables/part_4")

transaction = transaction1.union(transaction2).union(transaction3)

In [ ]:
# Join customers to transactions
transaction_consumer = transaction.join(consumer_list, on='user_id', how='left')
transaction_consumer.limit(3)

In [ ]:
consumer_no_transaction = consumer_list.join(transaction, on='user_id', how='left_anti')
print(f"Number of consumers that have not made a transaction: {consumer_no_transaction.count():,}")

**Joining customer transaction to merchant**

In [ ]:
# Correct for any multiple entries in consumer_fp
consumer_fp = consumer_fp.groupBy('order_datetime', 'user_id').agg(F.max('fraud_probability').alias('fraud_probability'))

# Add consumer fraud to transactions
final_df = transaction_consumer.join(consumer_fp, on=['order_datetime', 'user_id'], how='left').withColumnRenamed('fraud_probability', 'consumer_fraud')
no_fraud = final_df.filter(F.col("consumer_fraud").isNull()).count()
print(f"Number of transactions with no consumer fraud: {no_fraud:,}")

In [ ]:
# Correct for any multiple entries in merchant_fp
merchant_fp = merchant_fp.groupBy('order_datetime', 'merchant_abn').agg(F.max('fraud_probability').alias('fraud_probability'))

# Add merchant fraud to transactions by merchant and date
final_df = final_df.join(merchant_fp, on=['merchant_abn','order_datetime'], how = 'left').withColumnRenamed('fraud_probability', 'merchant_fraud')
no_fraud = final_df.filter(F.col("merchant_fraud").isNull()).count()
print(f"Number of transactions with no merchant fraud: {no_fraud:,}")

In [ ]:
# Impute all null fraud probabilities as 0
final_df = final_df.fillna(0, subset=['merchant_fraud', 'consumer_fraud'])
no_fraud = final_df.filter((final_df["consumer_fraud"]==0) & (final_df["merchant_fraud"]==0)).count()
print(f"Number of transactions with no merchant fraud or consumer fraud: {no_fraud:,}")

In [ ]:
# Save transaction-merchant-consumer data to file
final_df.write.mode('overwrite').parquet('../data/curated/fraud_watch/')

# Prepare external datasets
Here, we estimate weekly disposible income based on the difference between total_personal_income and the average spent on rent or morgage repayments per week. The calculation uses weekly variables as follows.
$$\text{weekly disposible income} = \text{total personal income} - (\text{median rent} \times \text{proportion of renters}) - (\text{median morgage repayment} \times \text{proportion of mortgage holders})$$

In [ ]:
# Download housing data
file_path = '../data/tables/sa2_dataset/main/C21_G37_SA2.csv'
url = "https://api.data.abs.gov.au/data/C21_G37_SA2/1+2+R_T+_T...SA2..2021.?detail=full"
headers = {'accept': 'text/csv'}

# Request data
response = requests.get(url, headers=headers, stream=True)

with open(file_path, 'wb') as file:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)

# Read in data and cast types
data = pd.read_csv(file_path, dtype={"REGION": object})

variables = {"R_T":'renting',
             "2":'owned_mortgage',
             "1": 'owned_outright',
             "_T":'total_responses',
             "REGION": 'sa2_code'}

# Aggregate to ignore the 'dwelling type' feature
tenure_data = data.groupby(['REGION', 'TENLLD']).agg('sum').reset_index()[['REGION', 'TENLLD', 'OBS_VALUE']]
tenure_data = tenure_data.pivot(index='REGION', columns='TENLLD', values='OBS_VALUE').reset_index()
tenure_data = tenure_data.rename(variables, axis=1)
tenure_data.head(5)

In [ ]:
# Apply calculations
tenure_data['percent_mortgage'] = tenure_data['owned_mortgage'] / tenure_data['total_responses']
tenure_data['percent_rent'] = tenure_data['renting'] / tenure_data['total_responses']

# Investigate number of records with missing data
zero_responses = tenure_data[tenure_data.isna().any(axis=1)]
print('Number of SA2 zones with no data from ABS: ', len(zero_responses))
zero_responses.head(5)

# Handle missing null values by setting to zero
percentage_tenure = tenure_data.fillna(0, axis=1).iloc[:,[0, -1, -2]]

In [ ]:
# Read in dataset with median statistics
variables = {1: "median_age", 
             2: "median_total_personal_income",
             3: "median_total_family_income",
             4: "median_total_household_income",
             5: "median_mortgage_repayment",
             6: "median_rent",
             7: "avg_people_per_bedroom",
             8: "avg_household_size"}

# Read in data
medians = pd.read_csv("../data/curated/sa2_dataset/C21_G02_SA2_clean.csv")

# Restructure table
medians = medians.pivot(index='sa2_code', columns=['type_of_value_code'], values='obs_value').reset_index().rename(columns=variables)
medians.columns.name = None
medians['sa2_code'] = medians.sa2_code.astype(str)
medians.head(3)

Before joining housing data to this table of medians, we investigate the records and notice that there are records that have null median summaries.

In [ ]:
# Read in list of SA2 codes and associated names
col_types = {"POSTCODE": str, "SA2_CODE_2021":str, "RATIO_FROM_TO": float}
sa2_names = pd.read_excel("../data/tables/correspondence/CG_POSTCODE_2021_SA2_2021.xlsx", converters=col_types)[['SA2_CODE_2021', 'SA2_NAME_2021', 'POSTCODE']]

# Find records with null columns
null_regions = medians[medians.isna().any(axis=1)]
null_regions = null_regions.merge(sa2_names, left_on='sa2_code', right_on='SA2_CODE_2021')

# Show the name of regions associated with null values 
null_regions.iloc[:,-2:]

We see that the external dataset is only missing values for SA2 regions marked as having "no usual address", and this is ok since they are special purpose codes that don't correspond with a SA2 zone.

In [ ]:
WEEKS_IN_MONTH = 4.345

# Add engineered housing features to ABS demographic data
abs_df = medians.merge(percentage_tenure, on='sa2_code')

# Calculate average weekly spending on housing per SA2 zone
# note: monthly mortgage repayment converted to weekly by dividing by # weeks in a month
abs_df['avg_housing_weekly'] = abs_df.median_rent*abs_df.percent_rent + abs_df.median_mortgage_repayment*(abs_df.percent_mortgage/WEEKS_IN_MONTH)
abs_df['weekly_personal_disposable'] = abs_df.median_total_personal_income - abs_df.avg_housing_weekly

# Associate postcodes to SA2 data using "correspondence.parquet"
sa2_names = pd.read_parquet('../data/curated/correspondence.parquet')
abs_df = sa2_names.merge(abs_df, on='sa2_code', how='left')

# Remove unecessary columns
abs_df = abs_df.drop(['median_mortgage_repayment', 'median_rent', 'percent_rent', 
                      'percent_mortgage', 'avg_housing_weekly', 
                      'median_total_personal_income', 'avg_people_per_bedroom'], axis=1)
abs_df.head(4)

In [ ]:
# Save to file
abs_df.to_parquet('../data/curated/sa2_dataset/abs_medians.parquet')

# Join ABS and customer data

In [ ]:
# Read in abs demographic data and customer/transaction data
abs_df = spark.read.parquet("../data/curated/sa2_dataset/abs_medians.parquet")

try:
    final_df = spark.read.parquet("../data/curated/fraud_watch")
except:
    final_df = final_df

# Join ABS data to transactions records
customer_details_abs = final_df.join(abs_df, on='postcode', how='left')

# Check NULL values for transaction data
customer_details_abs.summary('count').toPandas().to_dict(orient='records')[0]

We see that there are some customer records that are to locations that have no population data. Many of these postcodes are for PO boxes and LVR addresses. For this, we create another field `is_po_box` to help filter out transactions to users delivering to special postcodes.

The ranges for LVRs and PO Boxes were sourced from [Matthew Procter's Australian Postcodes website](https://www.matthewproctor.com/australian_postcodes#downloadlinks).

In [ ]:
# Deine PO/LVR range
po_box_codes = [("1000", "1999"), ("0200", "0299"), ("8000", "8999"),
                ("9000", "9999"), ("5800", "5999"), ("6800", "6999"),
                ("7800", "7999"), ("0900", "0999")]

# Determine if postcode is in a PO box/LVR range
po_box_condition = F.lit(False)
for lower, upper in po_box_codes:
    po_box_condition = po_box_condition | ((F.col("postcode") >= lower) & (F.col("postcode") <= upper))

# Apply condition
customer_details_abs = customer_details_abs.withColumn('is_po_box', po_box_condition)

# Confirm that transactions with no associated ABS data delivers to a PO box
customer_details_abs.filter("sa2_code IS NOT NULL AND is_po_box IS NULL").count()

In [ ]:
# Specify column order to see features better
col_order = ["order_id", "user_id","merchant_abn", "order_datetime", "dollar_value", 
              "postcode", "merchant_fraud", "consumer_fraud","weekly_personal_disposable",
              "median_total_household_income", "median_total_family_income"]

# Save changes to "all_details"
customer_details_abs.select(*col_order, *(set(customer_details_abs.columns) ^ set(col_order)))\
    .write.mode('overwrite').parquet('../data/curated/all_details/')

In [ ]:
temp = spark.read.parquet('../data/curated/all_details/')

# view sample of a random user's transaction history
temp.filter(F.col('user_id')==22957).orderBy('dollar_value', ascending=False).limit(5)